## 练习1 - 用数组实现队列结构

In [2]:

class ArrayQueue(object):
	def __init__(self, initSize):
		if initSize < 0:
			raise IllegalArgmentException("The init size is less than 0")

		self.items = [0] * initSize
		self.size = 0
		self.first = 0
		self.last = 0

	def push(self, item):
		if self.size == len(self.items):
			raise ArrayIndexOutOfBoundsException("The queue is full!")
		
		self.size += 1
		self.items[self.last] = item
		self.last = 0 if self.last == len(self.items) - 1 else self.last + 1

	def pull(self):
		if self.size == 0:
			raise ArrayIndexOfBoundsException("The queue is empty!")

		self.size -= 1
		tmp = self.first
		self.first = 0 if self.first == len(self.items) - 1 else self.first + 1

		return self.items[tmp]

	def peak(self):
		if self.size == 0:
			raise ArrayIndexOfBoundsException("The queue is empty!")

		return self.items[self.first]



In [3]:
aq = ArrayQueue(10)
aq.push(1)
aq.push(2)
print(aq.pull())
print(aq.pull())
#print(aq.pull())

1
2


## 练习2 - 用数组实现栈结构

In [17]:

class Stack(object):
	def __init__(self):
		self.items = list()

	def push(self, item):
		self.items.append(item)

	def pop(self):
		return self.items.pop()

	def clear(self):
		del self.items[:]

	def empty(self):
		return self.size() == 0

	def size(self):
		return len(self.items)

	def peak(self):
		return self.items[self.size() - 1]


class ArrayStack(object):
	def __init__(self, initSize):
		if initSize < 0:
			raise IllegalArgmentException("The init size is less than 0!")

		self.items = [0] * initSize
		self.size = 0

	def push(self, item):
		if self.size == len(self.items):
			raise ArrayIndexOutOfBoundsException("The stack is full!")

		self.items[self.size] = item
		self.size += 1

	def pop(self):
		if self.size == 0:
			raise ArrayIndexOutOfBoundsException("The stack is empty!")

		self.size -= 1
		
		return self.items[self.size]

	def peak(self):
		if self.size == 0:
			return None

		return self.items[self.size - 1]

		





In [18]:
at = ArrayStack(10)

In [19]:
at.push(1)
at.push(2)
print(at.pop())

2


https://blog.csdn.net/qq_38165374/article/details/70197718

## 练习3 - 面向对象高级编程

### 创建了一个class的实例，给该实例绑定任何属性和方法

这就是动态语言的灵活性

In [6]:
class Student(object):
    pass

In [7]:
s = Student()
s.name = 'Michael' # 动态给实例绑定一个属性
print(s.name)

Michael


In [8]:
# 定义一个函数作为实例方法
def set_age(self, age):
    self.age = age

In [11]:
from types import MethodType

In [12]:
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法

In [13]:
s.set_age(25) # 调用实例方法
s.age

25

但是，给一个实例绑定的方法，对另一个实例是不起作用的：

s2 = Student() # 创建新的实例
s2.set_age(25) # 尝试调用方法

为了给所有实例都绑定方法，可以给class绑定方法：

In [15]:
def set_score(self, score):
    self.score = score

In [16]:
Student.set_score = MethodType(set_score, Student)

In [18]:
s2.set_score(100)
s2.score

100

给class绑定方法后，所有实例均可调用，通常情况下，上面的set_score方法可以直接定义在class中，但动态绑定允许我们在程序运行的过程中动态给class加上功能，这在静态语言中很难实现。

### 使用`__slots__` 限制实例的属性

但是，如果我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性。

为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的`__slots__`变量，来限制该class实例能添加的属性：

In [20]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

In [21]:
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'
s.age = 25 # 绑定属性'age'
s.score = 99 # 绑定属性'score'

AttributeError: 'Student' object has no attribute 'score'

由于'score'没有被放到`__slots__`中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。

使用`__slots__`要注意，`__slots__`定义的属性仅对当前类实例起作用，对继承的子类是不起作用的：

### 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把属性随便改

这显然不合逻辑。为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [22]:
class Student(object):

    def get_score(self):
         return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [24]:
s = Student()
s.set_score(1000)

ValueError: score must between 0 ~ 100!

但是，这样调用又略显复杂，没有直接用属性这么直接简单。
有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？

还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [25]:
class Student(object):

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [26]:
s = Student()
s.score = 60 # OK，实际转化为s.set_score(60)
s.score # OK，实际转化为s.get_score()

60

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [28]:
class Student(object):
    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self,value):
        self._birth=value

    @property
    def age(self):
        return 2018-self.birth

In [30]:
s = Student()
s.birth = 2000
s.age
#s.age = 22

AttributeError: can't set attribute

https://www.cnblogs.com/284628487a/p/5588884.html